In [15]:
import torch
import time
import math

In [ ]:
#pytorch cpu

n = 100
m1 = torch.randn(n,n)
m2 = torch.randn(n,n)

start_time = time.time()
for i in range(100000):
  torch.matmul(m1,m2)

t1 = time.time() - start_time
print("pytorch cpu time 100x100 =",t1)

n += n
m1 = torch.randn(n,n)
m2 = torch.randn(n,n)

start_time = time.time()
for i in range(100000):
  torch.matmul(m1,m2)

t2 = time.time() - start_time
print("pytorch cpu time 200x200 =",t2)

x = t2 / t1
print(x,"times slower")

p = math.log(x) / math.log(2)

print("pytorch cpu -> 2^" + str(p))
